In [ ]:
import pickle
import os

from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

# silence warnings
import warnings
warnings.filterwarnings("ignore")

# operating system
import os

# time calculation to track some processes
import time

# numeric and matrix operations
import numpy as np

# loading ploting libraries
import matplotlib.pyplot as plt
%matplotlib inline

# python core library for machine learning and data science
import sklearn
from sklearn import set_config
set_config(transform_output = "pandas")

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.cluster import KMeans

In [ ]:
import os
import pickle
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Master Data Science/TFM/forecasting_df_test_2_weeks')

Este df corresponde a los últimos 4 meses.

In [ ]:
df.head()

,id,yearweek,date,item,category,department,store_code,store,region,price_per_unit,event_week_flag,product_cluster,store_cluster,week_of_year,month,quarter,year,units_sold,forecasted_units_sold
0,SUPERMARKET_3_367_PHI_3,201551,2015-12-20,SUPERMARKET_3_367,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,3.216,0,5,1,51,12,4,2015,14.0,10.0
1,SUPERMARKET_3_368_BOS_1,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_1,South_End,Boston,7.176,0,2,1,51,12,4,2015,27.0,15.0
2,SUPERMARKET_3_368_BOS_2,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_2,Roxbury,Boston,7.176,0,2,1,51,12,4,2015,36.0,29.0
3,SUPERMARKET_3_368_BOS_3,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_3,Back_Bay,Boston,7.176,0,2,1,51,12,4,2015,12.0,3.0
4,SUPERMARKET_3_368_NYC_1,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,NYC_1,Greenwich_Village,New York,7.176,0,2,1,51,12,4,2015,11.0,3.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 553031 entries, 0 to 553030
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   id                     553031 non-null  object        
 1   yearweek               553031 non-null  int64         
 2   date                   553031 non-null  datetime64[ns]
 3   item                   553031 non-null  object        
 4   category               553031 non-null  object        
 5   department             553031 non-null  object        
 6   store_code             553031 non-null  object        
 7   store                  553031 non-null  object        
 8   region                 553031 non-null  object        
 9   price_per_unit         553031 non-null  float64       
 10  event_week_flag        553031 non-null  int64         
 11  product_cluster        553031 non-null  int64         
 12  store_cluster          553031 non-null  int6

In [ ]:
df['store_cluster'].value_counts()

,count
store_cluster,
1,331819
3,110606
2,110606


In [ ]:
df['category'].value_counts()

,count
category,
SUPERMARKET,263231
HOME_&_GARDEN,188280
ACCESORIES,101520


#Calculamos el stock remanente y el revenue remanente

## Asumimos que el COG (coste de adquisicion de los productos) era de un 70% sobre el precio de venta final (la media de las empresas retail en EEUU). Por lo tanto el beneficio neto será el 30% del valor finnal de venta.

In [ ]:
df['difference_unit_sold'] = df['forecasted_units_sold'] - df['units_sold']

In [ ]:
df['difference_revenue'] = df['difference_unit_sold'] * df['price_per_unit']

In [ ]:
df

,id,yearweek,date,item,category,department,store_code,store,region,price_per_unit,...,product_cluster,store_cluster,week_of_year,month,quarter,year,units_sold,forecasted_units_sold,difference_unit_sold,difference_revenue
0,SUPERMARKET_3_367_PHI_3,201551,2015-12-20,SUPERMARKET_3_367,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,3.216,...,5,1,51,12,4,2015,14.0,10.0,-4.0,-12.864
1,SUPERMARKET_3_368_BOS_1,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_1,South_End,Boston,7.176,...,2,1,51,12,4,2015,27.0,15.0,-12.0,-86.112
2,SUPERMARKET_3_368_BOS_2,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_2,Roxbury,Boston,7.176,...,2,1,51,12,4,2015,36.0,29.0,-7.0,-50.232
3,SUPERMARKET_3_368_BOS_3,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_3,Back_Bay,Boston,7.176,...,2,1,51,12,4,2015,12.0,3.0,-9.0,-64.584
4,SUPERMARKET_3_368_NYC_1,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,NYC_1,Greenwich_Village,New York,7.176,...,2,1,51,12,4,2015,11.0,3.0,-8.0,-57.408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553026,SUPERMARKET_3_827_NYC_3,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,NYC_3,Tribeca,New York,1.200,...,3,1,16,4,2,2016,38.0,41.0,3.0,3.600
553027,SUPERMARKET_3_827_NYC_4,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,NYC_4,Brooklyn,New York,1.200,...,3,3,16,4,2,2016,22.0,11.0,-11.0,-13.200
553028,SUPERMARKET_3_827_PHI_1,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_1,Midtown_Village,Philadelphia,1.200,...,3,2,16,4,2,2016,15.0,24.0,9.0,10.800
553029,SUPERMARKET_3_827_PHI_2,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_2,Yorktown,Philadelphia,1.200,...,3,2,16,4,2,2016,12.0,19.0,7.0,8.400


## Aquí vemos que si las ventas son mayores a las predecidas, y son sobrantes de stock, los losses netos serán las unidades sobrantes por el precio unitario por el COG (0.7). Y la pérdida de oportunidad será que si las ventas sonmenores a las predecidas, y podríamos haber vendido más si hubiésemos tenido más stock, la pérdida de oportunidad será las unidades que podríamos haber vendido por el precio unitario por el beneficio neto de cada venta (0.3).

In [ ]:
# Crear la columna 'Losses'
df['Losses'] = np.where(df['difference_unit_sold'] > 0,
                        df['difference_unit_sold'] * df['price_per_unit'] * 0.7,
                        0)
# Crear la columna 'opportunity_loss'
df['opportunity_loss'] = np.where(df['difference_unit_sold'] < 0,
                                  df['difference_unit_sold'] * df['price_per_unit'] * 0.3,
                                  0)

In [ ]:
df['Losses'].sum()

2619710.90196

In [ ]:
df['opportunity_loss'].sum()

-1324745.6161842854

In [ ]:
df["profit"] = df["price_per_unit"] * df["units_sold"] * 0.3

In [ ]:
df["profit"].sum()

6003612.2193471445

In [ ]:
df

,id,yearweek,date,item,category,department,store_code,store,region,price_per_unit,...,month,quarter,year,units_sold,forecasted_units_sold,difference_unit_sold,difference_revenue,Losses,opportunity_loss,profit
0,SUPERMARKET_3_367_PHI_3,201551,2015-12-20,SUPERMARKET_3_367,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,3.216,...,12,4,2015,14.0,10.0,-4.0,-12.864,0.00,-3.8592,13.5072
1,SUPERMARKET_3_368_BOS_1,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_1,South_End,Boston,7.176,...,12,4,2015,27.0,15.0,-12.0,-86.112,0.00,-25.8336,58.1256
2,SUPERMARKET_3_368_BOS_2,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_2,Roxbury,Boston,7.176,...,12,4,2015,36.0,29.0,-7.0,-50.232,0.00,-15.0696,77.5008
3,SUPERMARKET_3_368_BOS_3,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_3,Back_Bay,Boston,7.176,...,12,4,2015,12.0,3.0,-9.0,-64.584,0.00,-19.3752,25.8336
4,SUPERMARKET_3_368_NYC_1,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,NYC_1,Greenwich_Village,New York,7.176,...,12,4,2015,11.0,3.0,-8.0,-57.408,0.00,-17.2224,23.6808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553026,SUPERMARKET_3_827_NYC_3,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,NYC_3,Tribeca,New York,1.200,...,4,2,2016,38.0,41.0,3.0,3.600,2.52,0.0000,13.6800
553027,SUPERMARKET_3_827_NYC_4,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,NYC_4,Brooklyn,New York,1.200,...,4,2,2016,22.0,11.0,-11.0,-13.200,0.00,-3.9600,7.9200
553028,SUPERMARKET_3_827_PHI_1,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_1,Midtown_Village,Philadelphia,1.200,...,4,2,2016,15.0,24.0,9.0,10.800,7.56,0.0000,5.4000
553029,SUPERMARKET_3_827_PHI_2,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_2,Yorktown,Philadelphia,1.200,...,4,2,2016,12.0,19.0,7.0,8.400,5.88,0.0000,4.3200


# Ahora calcularemos las losses, opportunity losses y profits para cada cluster de tiendas. Que son nuestros 3 diferentes pilotos. El que utilizaremos finalmente será el conservador (cluster tiendas rezagadas).

## Cluster de tienda 1 ("Tiendas Consolidadas")

In [ ]:
df_1 = df[df['store_cluster'] == 1]

In [ ]:
df_1

,id,yearweek,date,item,category,department,store_code,store,region,price_per_unit,...,month,quarter,year,units_sold,forecasted_units_sold,difference_unit_sold,difference_revenue,Losses,opportunity_loss,profit
0,SUPERMARKET_3_367_PHI_3,201551,2015-12-20,SUPERMARKET_3_367,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,3.216,...,12,4,2015,14.0,10.0,-4.0,-12.864,0.00,-3.8592,13.5072
1,SUPERMARKET_3_368_BOS_1,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_1,South_End,Boston,7.176,...,12,4,2015,27.0,15.0,-12.0,-86.112,0.00,-25.8336,58.1256
2,SUPERMARKET_3_368_BOS_2,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_2,Roxbury,Boston,7.176,...,12,4,2015,36.0,29.0,-7.0,-50.232,0.00,-15.0696,77.5008
3,SUPERMARKET_3_368_BOS_3,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_3,Back_Bay,Boston,7.176,...,12,4,2015,12.0,3.0,-9.0,-64.584,0.00,-19.3752,25.8336
4,SUPERMARKET_3_368_NYC_1,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,NYC_1,Greenwich_Village,New York,7.176,...,12,4,2015,11.0,3.0,-8.0,-57.408,0.00,-17.2224,23.6808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553022,SUPERMARKET_3_827_BOS_2,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,BOS_2,Roxbury,Boston,1.200,...,4,2,2016,12.0,1.0,-11.0,-13.200,0.00,-3.9600,4.3200
553023,SUPERMARKET_3_827_BOS_3,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,BOS_3,Back_Bay,Boston,1.200,...,4,2,2016,5.0,9.0,4.0,4.800,3.36,0.0000,1.8000
553024,SUPERMARKET_3_827_NYC_1,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,NYC_1,Greenwich_Village,New York,1.200,...,4,2,2016,49.0,27.0,-22.0,-26.400,0.00,-7.9200,17.6400
553026,SUPERMARKET_3_827_NYC_3,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,NYC_3,Tribeca,New York,1.200,...,4,2,2016,38.0,41.0,3.0,3.600,2.52,0.0000,13.6800


In [ ]:
df_1['Losses'].sum()

1661596.9023699998

In [ ]:
df_1['opportunity_loss'].sum()

-800359.8738085714

In [ ]:
df_1["profit"].sum()

3767300.803585714

## Cluster tienda 2 ("Tiendas Emergentes")

In [ ]:
df_2 = df[df['store_cluster'] == 2]

In [ ]:
df_2

,id,yearweek,date,item,category,department,store_code,store,region,price_per_unit,...,month,quarter,year,units_sold,forecasted_units_sold,difference_unit_sold,difference_revenue,Losses,opportunity_loss,profit
8,SUPERMARKET_3_368_PHI_1,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,PHI_1,Midtown_Village,Philadelphia,7.176,...,12,4,2015,12.0,12.0,0.0,0.000,0.0000,0.0000,25.8336
9,SUPERMARKET_3_368_PHI_2,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,PHI_2,Yorktown,Philadelphia,7.176,...,12,4,2015,12.0,8.0,-4.0,-28.704,0.0000,-8.6112,25.8336
18,SUPERMARKET_3_369_PHI_1,201551,2015-12-20,SUPERMARKET_3_369,SUPERMARKET,SUPERMARKET_3,PHI_1,Midtown_Village,Philadelphia,4.776,...,12,4,2015,2.0,5.0,3.0,14.328,10.0296,0.0000,2.8656
19,SUPERMARKET_3_369_PHI_2,201551,2015-12-20,SUPERMARKET_3_369,SUPERMARKET,SUPERMARKET_3,PHI_2,Yorktown,Philadelphia,4.776,...,12,4,2015,0.0,1.0,1.0,4.776,3.3432,0.0000,0.0000
28,SUPERMARKET_3_370_PHI_1,201551,2015-12-20,SUPERMARKET_3_370,SUPERMARKET,SUPERMARKET_3,PHI_1,Midtown_Village,Philadelphia,8.376,...,12,4,2015,5.0,-1.0,-6.0,-50.256,0.0000,-15.0768,12.5640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553009,SUPERMARKET_3_825_PHI_2,201616,2016-04-24,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,PHI_2,Yorktown,Philadelphia,4.776,...,4,2,2016,14.0,10.0,-4.0,-19.104,0.0000,-5.7312,20.0592
553018,SUPERMARKET_3_826_PHI_1,201616,2016-04-24,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,PHI_1,Midtown_Village,Philadelphia,1.536,...,4,2,2016,9.0,10.0,1.0,1.536,1.0752,0.0000,4.1472
553019,SUPERMARKET_3_826_PHI_2,201616,2016-04-24,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,PHI_2,Yorktown,Philadelphia,1.536,...,4,2,2016,10.0,6.0,-4.0,-6.144,0.0000,-1.8432,4.6080
553028,SUPERMARKET_3_827_PHI_1,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_1,Midtown_Village,Philadelphia,1.200,...,4,2,2016,15.0,24.0,9.0,10.800,7.5600,0.0000,5.4000


In [ ]:
df_2['Losses'].sum()

528406.87637

In [ ]:
df_2['opportunity_loss'].sum()

-282114.7561585714

In [ ]:
df_2["profit"].sum()

1208382.7310614283

## Cluster tienda 3 ("Tiendas Rezagadas")

### Esta es la que da un resultado mas aproximado a las ventas reales, el stock remanente es mas bajo. El riesgo es menor.

In [ ]:
df_3 = df[df['store_cluster'] == 3]

In [ ]:
df_3

,id,yearweek,date,item,category,department,store_code,store,region,price_per_unit,...,month,quarter,year,units_sold,forecasted_units_sold,difference_unit_sold,difference_revenue,Losses,opportunity_loss,profit
5,SUPERMARKET_3_368_NYC_2,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,NYC_2,Harlem,New York,7.176,...,12,4,2015,17.0,17.0,0.0,0.000,0.0000,0.0000,36.5976
7,SUPERMARKET_3_368_NYC_4,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,NYC_4,Brooklyn,New York,7.176,...,12,4,2015,0.0,5.0,5.0,35.880,25.1160,0.0000,0.0000
15,SUPERMARKET_3_369_NYC_2,201551,2015-12-20,SUPERMARKET_3_369,SUPERMARKET,SUPERMARKET_3,NYC_2,Harlem,New York,4.776,...,12,4,2015,0.0,2.0,2.0,9.552,6.6864,0.0000,0.0000
17,SUPERMARKET_3_369_NYC_4,201551,2015-12-20,SUPERMARKET_3_369,SUPERMARKET,SUPERMARKET_3,NYC_4,Brooklyn,New York,4.776,...,12,4,2015,1.0,3.0,2.0,9.552,6.6864,0.0000,1.4328
25,SUPERMARKET_3_370_NYC_2,201551,2015-12-20,SUPERMARKET_3_370,SUPERMARKET,SUPERMARKET_3,NYC_2,Harlem,New York,8.376,...,12,4,2015,19.0,20.0,1.0,8.376,5.8632,0.0000,47.7432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553007,SUPERMARKET_3_825_NYC_4,201616,2016-04-24,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,NYC_4,Brooklyn,New York,4.776,...,4,2,2016,8.0,4.0,-4.0,-19.104,0.0000,-5.7312,11.4624
553015,SUPERMARKET_3_826_NYC_2,201616,2016-04-24,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,NYC_2,Harlem,New York,1.536,...,4,2,2016,11.0,6.0,-5.0,-7.680,0.0000,-2.3040,5.0688
553017,SUPERMARKET_3_826_NYC_4,201616,2016-04-24,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,NYC_4,Brooklyn,New York,1.536,...,4,2,2016,22.0,12.0,-10.0,-15.360,0.0000,-4.6080,10.1376
553025,SUPERMARKET_3_827_NYC_2,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,NYC_2,Harlem,New York,1.200,...,4,2,2016,22.0,24.0,2.0,2.400,1.6800,0.0000,7.9200


In [ ]:
df_3['Losses'].sum()

429707.12322

In [ ]:
df_3['opportunity_loss'].sum()

-242270.9862171429

In [ ]:
df_3["profit"].sum()

1027928.6846999998

# Finalmente, para nuestro piloto conservador, filtramos por cluster de tiendas rezagadas y categoría Supermarket (que son los productos perecederso que se traducen en pérdidas directas si no soon vendidos).

## Supermarket

In [ ]:
df_cons = df[(df['store_cluster'] == 3) & (df['category'] == 'SUPERMARKET')]

In [ ]:
df

,id,yearweek,date,item,category,department,store_code,store,region,price_per_unit,...,month,quarter,year,units_sold,forecasted_units_sold,difference_unit_sold,difference_revenue,Losses,opportunity_loss,profit
0,SUPERMARKET_3_367_PHI_3,201551,2015-12-20,SUPERMARKET_3_367,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,3.216,...,12,4,2015,14.0,10.0,-4.0,-12.864,0.00,-3.8592,13.5072
1,SUPERMARKET_3_368_BOS_1,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_1,South_End,Boston,7.176,...,12,4,2015,27.0,15.0,-12.0,-86.112,0.00,-25.8336,58.1256
2,SUPERMARKET_3_368_BOS_2,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_2,Roxbury,Boston,7.176,...,12,4,2015,36.0,29.0,-7.0,-50.232,0.00,-15.0696,77.5008
3,SUPERMARKET_3_368_BOS_3,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,BOS_3,Back_Bay,Boston,7.176,...,12,4,2015,12.0,3.0,-9.0,-64.584,0.00,-19.3752,25.8336
4,SUPERMARKET_3_368_NYC_1,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,NYC_1,Greenwich_Village,New York,7.176,...,12,4,2015,11.0,3.0,-8.0,-57.408,0.00,-17.2224,23.6808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553026,SUPERMARKET_3_827_NYC_3,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,NYC_3,Tribeca,New York,1.200,...,4,2,2016,38.0,41.0,3.0,3.600,2.52,0.0000,13.6800
553027,SUPERMARKET_3_827_NYC_4,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,NYC_4,Brooklyn,New York,1.200,...,4,2,2016,22.0,11.0,-11.0,-13.200,0.00,-3.9600,7.9200
553028,SUPERMARKET_3_827_PHI_1,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_1,Midtown_Village,Philadelphia,1.200,...,4,2,2016,15.0,24.0,9.0,10.800,7.56,0.0000,5.4000
553029,SUPERMARKET_3_827_PHI_2,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_2,Yorktown,Philadelphia,1.200,...,4,2,2016,12.0,19.0,7.0,8.400,5.88,0.0000,4.3200


In [ ]:
df_cons['Losses'].sum()

206837.9784

In [ ]:
df_cons['opportunity_loss'].sum()

-125249.26165714287

In [ ]:
df_cons["profit"].sum()

552837.4477714285

# Y hacemos lo mismo con Supermarket 3, que son los productos más perecederos.


## Supermarket 3

In [ ]:
df_cons_3 = df[(df['store_cluster'] == 3) & (df['department'] == 'SUPERMARKET_3')]

In [ ]:
df_cons_3

,id,yearweek,date,item,category,department,store_code,store,region,price_per_unit,...,month,quarter,year,units_sold,forecasted_units_sold,difference_unit_sold,difference_revenue,Losses,opportunity_loss,profit
5,SUPERMARKET_3_368_NYC_2,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,NYC_2,Harlem,New York,7.176,...,12,4,2015,17.0,17.0,0.0,0.000,0.0000,0.0000,36.5976
7,SUPERMARKET_3_368_NYC_4,201551,2015-12-20,SUPERMARKET_3_368,SUPERMARKET,SUPERMARKET_3,NYC_4,Brooklyn,New York,7.176,...,12,4,2015,0.0,5.0,5.0,35.880,25.1160,0.0000,0.0000
15,SUPERMARKET_3_369_NYC_2,201551,2015-12-20,SUPERMARKET_3_369,SUPERMARKET,SUPERMARKET_3,NYC_2,Harlem,New York,4.776,...,12,4,2015,0.0,2.0,2.0,9.552,6.6864,0.0000,0.0000
17,SUPERMARKET_3_369_NYC_4,201551,2015-12-20,SUPERMARKET_3_369,SUPERMARKET,SUPERMARKET_3,NYC_4,Brooklyn,New York,4.776,...,12,4,2015,1.0,3.0,2.0,9.552,6.6864,0.0000,1.4328
25,SUPERMARKET_3_370_NYC_2,201551,2015-12-20,SUPERMARKET_3_370,SUPERMARKET,SUPERMARKET_3,NYC_2,Harlem,New York,8.376,...,12,4,2015,19.0,20.0,1.0,8.376,5.8632,0.0000,47.7432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553007,SUPERMARKET_3_825_NYC_4,201616,2016-04-24,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,NYC_4,Brooklyn,New York,4.776,...,4,2,2016,8.0,4.0,-4.0,-19.104,0.0000,-5.7312,11.4624
553015,SUPERMARKET_3_826_NYC_2,201616,2016-04-24,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,NYC_2,Harlem,New York,1.536,...,4,2,2016,11.0,6.0,-5.0,-7.680,0.0000,-2.3040,5.0688
553017,SUPERMARKET_3_826_NYC_4,201616,2016-04-24,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,NYC_4,Brooklyn,New York,1.536,...,4,2,2016,22.0,12.0,-10.0,-15.360,0.0000,-4.6080,10.1376
553025,SUPERMARKET_3_827_NYC_2,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,NYC_2,Harlem,New York,1.200,...,4,2,2016,22.0,24.0,2.0,2.400,1.6800,0.0000,7.9200


In [ ]:
df_cons_3['Losses'].sum()

115249.07999999999

In [ ]:
df_cons_3['opportunity_loss'].sum()

-74365.5209142857

In [ ]:
df_cons_3["profit"].sum()

334616.5686857143